In [241]:
import re
import pandas as pd
import numpy as np

In [255]:
# Copie du tableau : https://www.lemonde.fr/les-decodeurs/article/2024/07/02/legislatives-2024-pour-faire-barrage-au-rn-195-candidats-et-candidates-se-sont-deja-desistes-suivez-le-decompte-en-temps-reel_6245837_4355771.html
with open("LeMonde-Desistement.txt") as f:
    raw = f.read().splitlines()

In [256]:
# Renommage pour coller à "results_legislatives_2024_T1.csv"
raw = [r.replace('Guilaume Roit-Levêque', 'Guillaume Roit-Levêque') for r in raw]
raw = [r.replace('Hella Kribi-Romdhane', 'Hella Kribi - Romdhane') for r in raw]

In [257]:
# Parsing par circonscription
circo = []
end = [-1] + list(np.where(np.array(raw)=='Détails')[0])
for i in range(len(end)-1):
    circo.append(raw[end[i]+1:end[i+1]])

In [258]:
# Parsing des noms de circonscription
nom_circo = [(c[0], re.sub("[^0-9]", "", c[1])) for c in circo]
# Renommage pour coller à "departement_2022.csv"
nom_circo = [(c[0].replace("Français de l'étranger", "Français établis hors de France"), c[1]) for c in nom_circo]
nom_circo = [(c[0].replace("Saint-Martin et Saint-Barthélémy", "Saint-Martin/Saint-Barthélemy"), c[1]) for c in nom_circo]
nom_circo = [(c[0].replace("Wallis et Futuna", "Wallis-et-Futuna"), c[1]) for c in nom_circo]
nom_circo = [(c[0].replace("Polynésie française", "Polynésie Française"), c[1]) for c in nom_circo]

In [259]:
# Parsing des données par candidat
circo_data = {}
for i in range(len(circo)):
    c = circo[i]
    N = len(c[2:])//2
    candidats = []
    for j in range(N):
        candidat = {}
        nom = c[j+2+N].replace('désist.','')
        candidat['Nom'] = nom
        candidat['Nuance'] = c[j+2]
        candidat['Desistement'] = ('désist.' in c[j+2+N])
        candidats.append(candidat)
    circo_data[nom_circo[i]] = candidats

In [261]:
# Correspondance avec le tableau "results_legislatives_2024_T1.csv"
data_gouv = pd.read_csv("results_legislatives_2024_T1.csv", index_col=0)
list_candidats = data_gouv['Liste des candidats'].to_list()
list_candidats = [' '.join(c.split(' ')[1:]) for c in list_candidats]
list_candidats_up = [c.upper() for c in list_candidats]

In [262]:
# Ajout de données au tableau "results_legislatives_2024_T1.csv" et à la liste du Monde
for circo in circo_data:
    candidats = circo_data[circo]
    for candidat in candidats:
        idx = list_candidats_up.index(candidat['Nom'].upper())
        data = data_gouv.iloc[idx]
        candidat['Nom'] = data['Liste des candidats']
        candidat['Voix'] = data['Voix']
        candidat['% Inscrits'] = data['% Inscrits']
        candidat['% Exprimés'] = data['% Exprimés']
        if candidat['Desistement']: data_gouv.loc[idx, 'Elu(e)'] = 'DESISTE'

In [263]:
# Sauvegarde du tableau "results_legislatives_2024_T1.csv" avec les mentions de Désistement
data_gouv.to_csv("results_legislatives_2024_T1.csv")